# 법령 문서를 벡터 DB에 저장하고, 질문에 맞는 문서를 찾아 LLM에게 전달해서 답변을 생성하는 기본 RAG 시스템 구축
1. 문서의 내용을 읽고 쪼갠다
2. 임베딩 -> 벡터 데이터베이스에 저장
3. 사용자 질문에 대한 벡터 데이터베이스에서의 유사도 검색 수행
4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
5. keyword 사전 활용하여 일상 용어를 법령 용어로 바꾸어 검색 정확도 높이기 

# 1. 문서의 내용을 읽고 쪼개기 - Knowledge Base 구성을 위한 데이터 생성
- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
  - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
  - 비용 감소와 답변 생성시간 감소의 효과
  - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 한 조각 최대 1500자 
    chunk_overlap=200 # 앞뒤 200자씩 겹치게 함 (문맥이 끊기지 않도록)
)

loader = Docx2txtLoader('./tax_with_markdown.docx') 
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
document_list[52]

Document(metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한

# 2. 임베딩 모델 생성 및 벡터 DB 저장
- 임베딩이란? 텍스트를 숫자 배열로 반환하는 것 (이를 통해 '의미 유사도' 계산이 가능함)
- Chroma를 통한 벡터 DB 저장

In [4]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv() # 파일에서 API KEY 불러오기 

embedding = UpstageEmbeddings(model='solar-embedding-1-large') # 임베딩 모델 생성 

In [5]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
#database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 3. 사용자 질문에 대한 벡터 데이터베이스에서의 유사도 검색 수행
- 질문 쿼리를 벡터화하여 Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

[Document(id='002280b2-0edd-4633-b2fd-d063cc0593b7', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용
- RetrievalQA를 통해 LLM에 전달
  - RetrievalQA는 create_retrieval_chain으로 대체됨
  - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [7]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,                                # ChatUpstage
    retriever=database.as_retriever(),  # 검색기
    chain_type_kwargs={"prompt": prompt}
)

In [10]:
ai_message = qa_chain.invoke({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 3,250,000원입니다. 이는 5천만원 초과 8,800만원 이하의 종합소득 과세표준에 해당하는 세율인 24%를 적용한 금액에서 누진공제액 624만원을 차감한 금액입니다.'}

# 5. keyword 사전 활용하여 일상 용어를 법령 용어로 바꾸어 검색 정확도 높이기 
- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 오직 질문만 그대로 리턴해주세요 다른 말은 절대 포함하지 마세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [12]:
new_question = dictionary_chain.invoke({"question": query})

In [13]:
new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [14]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [15]:
ai_response = tax_chain.invoke({"question": query})
ai_response

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 5,000만원 초과 8,800만원 이하의 세율을 적용하여 계산합니다. 따라서 624만원 + (5,000만원을 초과하는 금액인 3,000만원의 24%)를 계산하면 세액은 1,272만원이 됩니다.'}